# Overview

## Objective

This notebook provides an example of how to registry a Tensoflow model on SAS Model Manager.

The goal is manage the e2e with the model deployment on RedHat OpenShift

## Assumption

At that time, SAS does not have a proper Python library to deal with Tensorflow models.

But we can use some pzmm and sasctl functionalities.